In [80]:
from mpl_toolkits.axes_grid1 import ImageGrid
import matplotlib.pyplot as plt
from PIL import Image
def DisplaySimilarImage(images):
        res = []
        fig = plt.figure(figsize=(100., 100.))
        noRows = max(1, len(images)//4)
        grid = ImageGrid(fig,111,
                         nrows_ncols=(noRows, 10//noRows),
                         axes_pad=0.1,
                         )
        for path in images:
            img = Image.open(path)
            img = img.resize((150, 150), resample=Image.BILINEAR)
            res.append(img)
    
        

        for ax, im in zip(grid, res):
            ax.imshow(im)
            ax.axis('off')

        _ = plt.show()

<h1> Random Generator </h1>

In [36]:
import os
import random
import pandas as pd
def create_random(rand):
    def find_files(directory):
        file_paths = []
        for root, dirs, files in os.walk(directory):
            for file in files:
                if len(file.split(".")[0]) != 3:
                    continue
                file_paths.append(os.path.join(root, file))
        file_paths.sort()
        return file_paths

    def random_files(file_paths, num_files):
        if num_files > len(file_paths):
            print("Not enough files to select the desired number. Selecting all files.")
            return file_paths
        return random.sample(file_paths, num_files)

    directory = 'images'
    all_files = find_files(directory)

    number_of_files_to_select = rand

    selected_files = random_files(all_files, number_of_files_to_select)
    dfTxt = pd.read_csv("dataset/SBU_captioned_photo_dataset_captions.txt", sep="\t", header=None)
    resLine = []
    for path in selected_files:
        folderNum = int(path.split("/")[1])
        fileNum = int(path.split("/")[2].split(".")[0])
        if folderNum != 0:
            line = (folderNum)*1000 + fileNum 
            resLine.append(dfTxt[0].iloc[line])
        else:
            line = fileNum
            resLine.append(dfTxt[0].iloc[line])


    dictPath = dict(zip(selected_files, resLine))
    return dictPath


In [37]:
dictPath = create_random(1000)

<h1> Recall with IndexFlatL2</h1>

In [29]:
import matplotlib.pyplot as plt
from PIL import Image
from mpl_toolkits.axes_grid1 import ImageGrid
import requests
from io import BytesIO
from collections import defaultdict
recallMatch = defaultdict(int)
import faiss
import torch
from transformers import AutoModel, AutoTokenizer
imgIndex = faiss.read_index('multi1MImageVectors.index')
model = AutoModel.from_pretrained("google/siglip-so400m-patch14-384")
tokenizer = AutoTokenizer.from_pretrained("google/siglip-so400m-patch14-384")


In [38]:

def recallK(k):
    model.eval()
    k = k
    res = []
    recall = {}
    fig = plt.figure(figsize=(100., 100.))
    for key, value in dictPath.items():
        inputs = tokenizer([value], padding="max_length", return_tensors="pt")
        with torch.no_grad():
            
            text_features = model.get_text_features(**inputs)
            similar_images = ImageSimilar(text_features, k)

        for idx, img_path in enumerate(similar_images):
            if img_path == key:
                recallMatch[img_path] = idx
    i = 1            
    while(i<k+1):
        count = 0
        for key, value in recallMatch.items():
            if value == 0:
                continue
            elif value <= i:
                count+=1
        #print(count)
        recall[i] = count/10
        if i == 1:
            i+=9
        else:
            i+=10
    return recall
            
            
                
def indices_to_images(indices):
    image_paths = []
    for i in indices:
        folder_name = i // 1000
        file_name = i % 1000
        folder_str = f"{folder_name:04d}"
        file_str = f"{file_name:03d}.jpg"
        image_paths.append("images" + "/" + folder_str + "/" + file_str)
    return image_paths
        
def ImageSimilar(features, k):
    _, I = imgIndex.search(features, k) 
    similarList = I.tolist()
    similarPath = indices_to_images(similarList[0])
    return similarPath
            
    
    


In [39]:
print(recallK(100))

{1: 8.8, 10: 35.3, 20: 47.8, 30: 56.9, 40: 63.1, 50: 67.4, 60: 71.7, 70: 76.6, 80: 80.9, 90: 84.1, 100: 87.0}


<Figure size 10000x10000 with 0 Axes>

<h1> BruteForce to IVF </h1>

In [13]:
import numpy as np
import faiss
index = faiss.read_index('multi1MImageVectors.index')
vectors = []
index.ntotal
for i in range(index.ntotal):
    vectors.append(index.reconstruct(i))
vectors = np.array(vectors)
index_flatl2 = faiss.IndexFlatL2(vectors.shape[1])
index_flatl2.add(vectors)
nlist = 50
kmeans = faiss.Kmeans(vectors.shape[1], nlist)
kmeans.train(vectors)
quantizer = faiss.IndexFlatL2(vectors.shape[1])
quantizer.add(kmeans.centroids)
index_ivfflat = faiss.IndexIVFFlat(quantizer, vectors.shape[1], nlist)
index_ivfflat.train(vectors)
index_ivfflat.add(vectors)

In [14]:
faiss.write_index(index_ivfflat, 'multi1MIVFImageVectors.index')

<h1> Recall with IVF <h1>

In [40]:
import matplotlib.pyplot as plt
from PIL import Image
from mpl_toolkits.axes_grid1 import ImageGrid
import requests
from io import BytesIO
from collections import defaultdict
recallMatch = defaultdict(int)
import faiss
from transformers import AutoModel, AutoTokenizer
import torch

imgIndex = faiss.read_index('multi1MIVFImageVectors.index')
#model = AutoModel.from_pretrained("google/siglip-so400m-patch14-384")
#tokenizer = AutoTokenizer.from_pretrained("google/siglip-so400m-patch14-384")

In [41]:
def recallK(k):
    model.eval()
    k = k
    res = []
    recall = {}
    fig = plt.figure(figsize=(100., 100.))
    for key, value in dictPath.items():
        inputs = tokenizer([value], padding="max_length", return_tensors="pt")
        with torch.no_grad():
            
            text_features = model.get_text_features(**inputs)
            similar_images = ImageSimilar(text_features, k)

        for idx, img_path in enumerate(similar_images):
            if img_path == key:
                recallMatch[img_path] = idx
    i = 1            
    while(i<k+1):
        count = 0
        for key, value in recallMatch.items():
            if value == 0:
                continue
            elif value <= i:
                count+=1
        #print(count)
        recall[i] = count/10
        if i == 1:
            i+=9
        else:
            i+=10
    return recall
            
            
                
def indices_to_images(indices):
    image_paths = []
    for i in indices:
        folder_name = i // 1000
        file_name = i % 1000
        folder_str = f"{folder_name:04d}"
        file_str = f"{file_name:03d}.jpg"
        image_paths.append("images" + "/" + folder_str + "/" + file_str)
    return image_paths
        
def ImageSimilar(features, k):
    _, I = imgIndex.search(features, k) 
    similarList = I.tolist()
    similarPath = indices_to_images(similarList[0])
    return similarPath
            

In [42]:
print(recallK(100))

{1: 0.7, 10: 1.6, 20: 2.2, 30: 2.9, 40: 3.2, 50: 3.4, 60: 3.6, 70: 3.6, 80: 3.7, 90: 3.8, 100: 4.1}


<Figure size 10000x10000 with 0 Axes>